# Load Pretrained Model 

In [1]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from nni.compression.pytorch.speedup import ModelSpeedup
from nni.compression.pytorch.utils import count_flops_params
import time

from mnist_model import Net, train, test, device, optimizer_scheduler_generator, trainer

# Load pretrained model
model = torch.load("mnist_cnn.pt")
model.eval()

# show the model stbructure, note that pruner will wrap the model layer.
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


### Performance and statistics of pre-trained model 

In [2]:
start = time.time()

pre_best_acc = test(model, device)
pre_test_time = time.time() - start

pre_flops, pre_params, _ = count_flops_params(model, torch.randn([3, 1, 28, 28]).to(device))
print(f'Pretrained model FLOPs {pre_flops/1e6:.2f} M, #Params: {pre_params/1e6:.2f}M, Accuracy: {pre_best_acc: .2f}%, Test-time: {pre_test_time: .4f}s')


Test set: Average loss: 0.0267, Accuracy: 9919/10000 (99.19%)

+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs   | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
|   0   | conv1 | Conv2d | (32, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 32, 26, 26) |  194688  |   320   |
|   1   | conv2 | Conv2d | (64, 32, 3, 3) | (3, 32, 26, 26) | (3, 64, 24, 24) | 10616832 |  18496  |
|   2   | fc1   | Linear |  (128, 9216)   |    (3, 9216)    |     (3, 128)    | 1179648  | 1179776 |
|   3   | fc2   | Linear |   (10, 128)    |     (3, 128)    |     (3, 10)     |   1280   |   1290  |
+-------+-------+--------+----------------+-----------------+-----------------+----------+---------+
FLOPs total: 11992448
#Params total: 1199882
Pretrained model FLOPs 11.99 M, #Params: 1.20M, Accuracy:  99.19%, Test-time:  1.75

# Pruning Model  Activation Mean Rank

In [3]:
from nni.compression.pytorch.pruning import ADMMPruner
from nni.compression.pytorch.pruning import ActivationMeanRankPruner
from nni.compression.pytorch.speedup import ModelSpeedup
import nni

import torch.nn.functional as F

def pruner_function(config_list):

    model = torch.load("mnist_cnn.pt")
    model.eval()

    traced_optimizer = nni.trace(optim.Adadelta)(model.parameters(), lr=1.0)
    criterion = F.nll_loss
    
    # Using ADMMPruner to prune the model and generate the masks.
    #pruner = ADMMPruner(model, config_list, trainer, traced_optimizer, criterion, iterations=5, training_epochs=1, granularity='coarse-grained')
    
    pruner = ActivationMeanRankPruner(model, config_list, trainer, traced_optimizer, criterion, training_batches=20)
    
    # show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
    #print(model)

    # compress the model and generate the masks
    _, masks = pruner.compress()

    # show the masks sparsity
    print("Showing the masks sparsity")
    for name, mask in masks.items():
        print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum() / mask['weight'].numel()))


    # need to unwrap the model, if the model is wrapped before speedup
    pruner._unwrap_model()

    # speedup the model, for more information about speedup, please refer :doc:`pruning_speedup`.
    ModelSpeedup(model, torch.rand(3, 1, 28, 28).to(device), masks).speedup_model()

    #print("Model after speedup")
    #print(model)

    optimizer, scheduler = optimizer_scheduler_generator(model)
    
    # fine- tuning model compacted model
    # tuning and evaluate the model on MNIST dataset
    total_epoch = 3
    
    for epoch in range(1, total_epoch + 1):
        train(model, device, optimizer=optimizer, epoch=epoch)
        test(model, device)
        scheduler.step()
        
    return model

In [4]:
def Perfomance_function(model):
    print("Model after speedup")
    print(model)
    
    start = time.time()
    best_acc = test(model, device)
    test_time = time.time() - start

    flops, params, _ = count_flops_params(model, torch.randn([3, 1, 28, 28]).to(device))

    print(f'Pretrained model FLOPs {pre_flops/1e6:.2f} M, #Params: {pre_params/1e6:.2f}M, Accuracy: {pre_best_acc: .2f}%, , Test-time: {pre_test_time: .4f}s')
    print(f'Finetuned model FLOPs {flops/1e6:.2f} M, #Params: {params/1e6:.2f}M, Accuracy: {best_acc: .2f}%, Test-time: {test_time: .4f}s, Speed-up: {pre_test_time/test_time: .2f}x')

## ADMM Configuration 1

In [5]:
config_list = [{
    'sparsity_per_layer': 0.50,
    'op_types': ['Linear', 'Conv2d']
}, {
    'exclude': True,
    'op_names': ['fc2']
}]


pruned_model = pruner_function(config_list=config_list)

[2022-10-03 20:16:40] WARNING: The old API trainer,traced_optimizer,criterion,training_batches,activation,mode,dummy_input will be deprecated after NNI v3.0, please using the new one evaluator,training_steps,activation,mode,dummy_input
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.001160
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.349021
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.076578
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.009604
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.008620
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.225321
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.177449
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.000831
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.089915
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.030283
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.061408
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.011798
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.066102
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.012305
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.029739
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.

/home/jakaria/.local/lib/python3.10/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


Train Epoch: 1 [0/60000 (0%)]	Loss: 1.774406
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.139781
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.145159
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.206474
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.237173
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.049166
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.112457
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.069230
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.089296
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.036387
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.060088
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.083245
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.029452
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.112076
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.199143
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.235638
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.115838
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.116069
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.124952
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.103599
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.137121
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.037120
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.263866
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.248862
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.008364
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.043392
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.016718
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.129828
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.080869
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.037364
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.023279
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.190189
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.047614
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.049677
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.032836
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.040715
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.014526
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.182779
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.014776
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.200945


In [6]:
Perfomance_function(pruned_model)

Model after speedup
Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

Test set: Average loss: 0.0359, Accuracy: 9887/10000 (98.87%)

+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
| Index | Name  |  Type  |  Weight Shape  |    Input Size   |   Output Size   |  FLOPs  | #Params |
+-------+-------+--------+----------------+-----------------+-----------------+---------+---------+
|   0   | conv1 | Conv2d | (16, 1, 3, 3)  |  (3, 1, 28, 28) | (3, 16, 26, 26) |  97344  |   160   |
|   1   | conv2 | Conv2d | (32, 16, 3, 3) | (3, 16, 26, 26) | (3, 32, 24, 24) | 2654208 |   4640  |
|   2   | fc1   | Linear |   (64, 4608)   |    (3, 4608)    |     (3, 64)    

## ADMM Configuration 2

In [7]:
config_list = [{
    'op_types': ['Conv2d'],
    'total_sparsity': 0.5
    }, {
    'op_names': ['Linear'],
    'total_sparsity': 0.8
    },
    {
    'exclude': True,
    'op_names': ['fc2']
}]


pruned_model = pruner_function(config_list=config_list)

[2022-10-03 20:17:43] WARNING: The old API trainer,traced_optimizer,criterion,training_batches,activation,mode,dummy_input will be deprecated after NNI v3.0, please using the new one evaluator,training_steps,activation,mode,dummy_input
[2022-10-03 20:17:43] WARNING: op_names ['Linear'] not found in model
[2022-10-03 20:17:43] WARNING: op_names ['Linear'] not found in model
[2022-10-03 20:17:43] WARNING: op_names ['Linear'] not found in model
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005200
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.178087
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.006849
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.030576
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.047069
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.389356
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.077931
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.009553
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.051433
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.041742
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.050278
Train Epoch: 1 [7040/60000 (

Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.255271
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.057768
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.013755
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.058269
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.069035
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.144401
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.049871
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.076872
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.207815
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.116002
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.113186
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.037944
Train Epoch: 1 [13440/60000 (22%)]	Loss: 0.080912
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.090280
Train Epoch: 1 [14720/60000 (25%)]	Loss: 0.122528
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.036701
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.161793
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.071117
Train Epoch: 1 [17280/60000 (29%)]	Loss: 0.001913
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.036648
Train E

Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.166766
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.016853
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.002031
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.012238
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.013285
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.085471
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.141067
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.048926
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.016826
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.035165
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.063727
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.024377
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.036193
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.038798

Test set: Average loss: 0.0305, Accuracy: 9905/10000 (99.05%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.048923
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.035140
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.017896
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.024733
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.000873


In [10]:
Perfomance_function(pruned_model)

Model after speedup
Net(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(4, 7, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1008, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

Test set: Average loss: 0.0498, Accuracy: 9841/10000 (98.41%)

+-------+-------+--------+--------------+----------------+----------------+--------+---------+
| Index | Name  |  Type  | Weight Shape |   Input Size   |  Output Size   | FLOPs  | #Params |
+-------+-------+--------+--------------+----------------+----------------+--------+---------+
|   0   | conv1 | Conv2d | (4, 1, 3, 3) | (3, 1, 28, 28) | (3, 4, 26, 26) | 24336  |    40   |
|   1   | conv2 | Conv2d | (7, 4, 3, 3) | (3, 4, 26, 26) | (3, 7, 24, 24) | 145152 |   259   |
|   2   | fc1   | Linear | (128, 1008)  |   (3, 1008)    |    (3, 128)    | 129024 |  129152 |
|   3   